In [22]:
import torch
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForMaskedLM, Trainer, TrainingArguments


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [24]:
dataset = load_dataset("code_search_net", "python",cache_dir="./Datasets")

In [25]:
dataset.column_names

{'train': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'test': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'validation': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url']}

In [26]:
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base", cache_dir="./Models")
model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base", cache_dir="./Models").to(device)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
def tokenize_code(code_snippet, tokenizer):
    return tokenizer(
        code_snippet, max_length=512, truncation=True, padding='max_length', return_tensors="pt"
    )

In [28]:
def preprocess_data(batch, tokenizer):
    batch["input_ids"] = tokenize_code(batch["func_code_string"], tokenizer)["input_ids"]
    batch["labels"] = tokenize_code(batch["func_documentation_string"], tokenizer)["input_ids"]
    return batch

In [29]:
train_dataset = dataset["train"].shuffle(seed=42).select(range(1000)).map(lambda x: preprocess_data(x, tokenizer), batched=True)
eval_dataset = dataset["validation"].shuffle(seed=42).select(range(200)).map(lambda x: preprocess_data(x, tokenizer), batched=True)

In [30]:
training_args = TrainingArguments(
    output_dir="./codebert-finetuned",    # output directory
    num_train_epochs=3,                   # number of epochs
    per_device_train_batch_size=16,       # batch size
    per_device_eval_batch_size=16,        # eval batch size
    warmup_steps=500,                     # warmup steps
    weight_decay=0.01,                    # weight decay
    logging_dir="./logs",                 # logging directory
    logging_steps=10,
    evaluation_strategy="steps",          # Evaluation after every logging step
    save_total_limit=2,                   # Keep only last two checkpoints
    save_steps=500,                       # Save model every 500 steps
    report_to="none",
    fp16=True                     # No reports (e.g., to wandb)
)

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:463: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [32]:
trainer.train()

  0%|          | 0/189 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  5%|▌         | 10/189 [04:44<1:24:23, 28.29s/it]

{'loss': 18.0067, 'grad_norm': 62.93367385864258, 'learning_rate': 7.000000000000001e-07, 'epoch': 0.16}


                                                  
  5%|▌         | 10/189 [06:35<1:24:23, 28.29s/it]

{'eval_loss': 17.82748794555664, 'eval_runtime': 110.4363, 'eval_samples_per_second': 1.811, 'eval_steps_per_second': 0.118, 'epoch': 0.16}


 11%|█         | 20/189 [11:19<1:24:03, 29.84s/it]

...

                                                   
 95%|█████████▌| 180/189 [2:02:37<04:15, 28.34s/it]

{'eval_loss': 1.5190330743789673, 'eval_runtime': 102.3456, 'eval_samples_per_second': 1.954, 'eval_steps_per_second': 0.127, 'epoch': 2.86}


100%|██████████| 189/189 [2:06:30<00:00, 40.16s/it]

{'train_runtime': 7590.0833, 'train_samples_per_second': 0.395, 'train_steps_per_second': 0.025, 'train_loss': 6.4713023755916215, 'epoch': 3.0}


TrainOutput(global_step=189, training_loss=6.4713023755916215, metrics={'train_runtime': 7590.0833, 'train_samples_per_second': 0.395, 'train_steps_per_second': 0.025, 'total_flos': 789796389888000.0, 'train_loss': 6.4713023755916215, 'epoch': 3.0})

In [33]:
model.save_pretrained("./codebert-finetuned")
tokenizer.save_pretrained("./codebert-finetuned")

('./codebert-finetuned\\tokenizer_config.json',
 './codebert-finetuned\\special_tokens_map.json',
 './codebert-finetuned\\vocab.json',
 './codebert-finetuned\\merges.txt',
 './codebert-finetuned\\added_tokens.json')

In [34]:
def generate_documentation(code_snippet, model, tokenizer):
    inputs = tokenizer(code_snippet, return_tensors="pt", padding=True, truncation=True).to(device)
    print(inputs)
    outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [35]:
code_example = """
def add_numbers(a, b):
    return a + b
"""

In [50]:
documentation = generate_documentation(code_example, model, tokenizer)
print("Generated Documentation:", documentation)

TypeError: The current model class (RobertaForMaskedLM) is not compatible with `.generate()`, as it doesn't have a language model head. Classes that support generation often end in one of these names: ['ForCausalLM', 'ForConditionalGeneration', 'ForSpeechSeq2Seq', 'ForVision2Seq'].

In [59]:
from transformers import RobertaTokenizer, RobertaForCausalLM
import torch

# Define the path to your saved model directory
saved_model_path = './codebert-finetuned-t5base/'

# Load the tokenizer and the fine-tuned model from the saved checkpoint
tokenizer = RobertaTokenizer.from_pretrained(saved_model_path)
model = RobertaForCausalLM.from_pretrained(saved_model_path, is_decoder=True)

# Set the model to eval mode for inference
model.eval()

# Ensure to use the correct device (GPU/TPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Example Python code to generate documentation for



RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

In [60]:
code_snippet = "def add_numbers(a, b):\n    return a + b"

# Tokenize the input code
inputs = tokenizer(code_snippet, return_tensors="pt", padding=True, truncation=True).to(device)

# Generate documentation using the fine-tuned model
with torch.no_grad():
    outputs = model.generate(inputs['input_ids'], max_length=300, num_beams=5, early_stopping=True)

# Decode the output to get the generated documentation
generated_doc = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Documentation:\n", generated_doc)

KeyboardInterrupt: 

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import shutil
import os

# Define device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Delete the existing cache directory to avoid loading corrupted files
cache_dir = "./Models"
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)

# Re-download the tokenizer and model, forcing a fresh download
tokenizer = T5Tokenizer.from_pretrained("Salesforce/codet5-base", cache_dir=cache_dir, force_download=True)
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base", cache_dir=cache_dir, force_download=True).to(device)


# Test the model and tokenizer by encoding and decoding
test_input = "def add_numbers(a, b): return a + b"
inputs = tokenizer(test_input, return_tensors="pt").to(device)
outputs = model.generate(inputs.input_ids)

# Decode the generated output
generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_code)


c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\sai\UCE\Code_doc_generator\Models\models--Salesforce--codet5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The tokenizer class you load from this checkpoint is not the same type as the class this

TypeError: not a string